In [4]:
from kmodes.kprototypes import KPrototypes
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [6]:
path = '/Users/mehmetdeniz/Library/CloudStorage/GoogleDrive-mehmetbakideniz@gmail.com/My Drive/meta_audience_insights_age_data.csv'
df=pd.read_csv(path)

In [ ]:
df=df[['clicks', 'cost', 'gender', 'age', 'campaign_type','purchase_value']]

In [ ]:
grouped = df.groupby(['gender', 'age', 'campaign_type'])

df2 = grouped.agg({
    'cost': 'sum',
    'clicks': 'sum',
    'purchase_value': 'sum'
}).reset_index()



In [ ]:
df2['cpc'] = np.where(df2['clicks'] != 0, df2['cost'] / df2['clicks'], 0)
df2['value/cost']=np.where(df2['cost'] != 0, df2['purchase_value'] / df2['cost'], 0)
df2.drop(['clicks', 'purchase_value'], axis=1, inplace= True )

In [ ]:
len(df2)

In [ ]:
df2['gender']=df2['gender'].astype('category')
df2['age']=df2['age'].astype('category')
df2['campaign_type']=df2['campaign_type'].astype('category')


In [ ]:
categorical_columns = df2.select_dtypes(include=['category']).columns
indexes = [df2.columns.get_loc(column) for column in categorical_columns]

In [ ]:
indexes

In [ ]:


numerical_columns = df2.select_dtypes(include=[float, int]).columns
# Instantiate the StandardScaler object and fit it on the selected numerical columns

# Instantiate the MinMaxScaler object
minmax_scaler = MinMaxScaler(feature_range=(-1, 1))

# Apply MinMaxScaler to the standardized numerical columns
df2[numerical_columns] = minmax_scaler.fit_transform(df2[numerical_columns])
# Revert back to the original values

#df2[numerical_columns] = pd.DataFrame(minmax_scaler.inverse_transform(df2[numerical_columns]), columns=numerical_columns)


In [ ]:
kp = KPrototypes(n_clusters=1, init='Huang', n_init=3, verbose=True)
kp.fit_predict(df2, categorical=indexes)



In [ ]:
print(kp.cost_)


In [ ]:
df2['labels']=kp.labels_
clusters=kp.cluster_centroids_

def calculate_distances(df, centroids):
    def euclidean_dissim(a, b):
        """Euclidean distance dissimilarity function"""
        return np.sum((a - b) ** 2)

    def matching_dissim(a, b):
        """Simple matching dissimilarity function"""
        return np.sum(a != b)

    # Initialize an empty list to store distance scores
    distances = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Get the corresponding centroid for the current row
        indx=row['labels']
        centroid = centroids[indx]
        row = row.iloc[:-1]
        categorical_values = []
        numerical_values = []
        for value in row:
            if isinstance(value, (str, pd.Categorical)):
                categorical_values.append(value)
            else:
                numerical_values.append(value)

        # Convert categorical and numerical sections to numpy arrays
        categorical_array = np.array(categorical_values)
        numerical_array = np.array(numerical_values)

        numeric_centroid = []
        categorical_centroid = []

        for value in centroid:
            try:
                numeric_value = float(value)
                numeric_centroid.append(numeric_value)
            except ValueError:
                categorical_centroid.append(value)

        numeric_centroid = np.array(numeric_centroid)
        categoric_centroid = categorical_centroid
        categorical_dissimilarity = matching_dissim(categorical_array, categoric_centroid)

        numerical_dissimilarity = euclidean_dissim(numerical_array, numeric_centroid)
        gamma=kp.gamma
        distance = gamma*categorical_dissimilarity + numerical_dissimilarity

        distances.append(distance)

    df['distance'] = distances

    return df



In [ ]:
kp.get_params

In [ ]:
df2 = calculate_distances(df2, clusters)


In [ ]:
df2[numerical_columns] = pd.DataFrame(minmax_scaler.inverse_transform(df2[numerical_columns]), columns=numerical_columns)


In [ ]:
df2.sort_values(by=['distance'], inplace=True, ascending=False)

In [ ]:
df2

In [ ]:
df2['labels'].value_counts()

In [ ]:
len(df2)

In [ ]:
df_anamoly=df2

In [ ]:
# Calculate the number of rows for the first 5%
num_rows = int(len(df_anamoly) * 0.2)

# Create the new DataFrame containing the first 5% of rows
df_anamoly = df_anamoly.head(num_rows)

In [ ]:
df_anamoly.reset_index(drop=True, inplace=True)

In [ ]:
df_anamoly